# Regressione Lineare con Spark MLlib

### In questo notebook vedremo come eseguire una semplice regressione, utilizzando un modello di Regressione Lineare, con il modulo MLlib di Spark. Il modello che andremo a creare avrà lo scopo di stimare il valore di un'abitazione utilizzando un set di proprietà:

### CRIM Tasso di criminalità per capita
### ZN Percentuale di terreni residenziali suddivisi in zone per lotti superiori a 25.000 sq.ft.
### INDUS Percentuale di ettari di attività non al dettaglio per città.
### CHAS Variabile dummy che indica la prossimità al fiume Charles.
### NOX Concentrazione di ossido d'azoto (parti per 10 milioni).
### RM Numero medio di stanze per abitazione
### AGE Percentuale di abitazione occupate costruite dopo il 1940
### DIS Media pesata delle distanze da 5 centri lavorativi di Boston.
### RAD Indice di accessibilità ad autostrade
### TAX Aliquota dell'imposta sulla proprietà a valore pieno in 10.000 USD.
### PRATIO Rapporto studente-insegnante per città.
### BLACK 1000(Bk - 0.63)^2 dove Bk è la percentuale di abitanti di colore per città
### LSTAT Percentuale della popolazione povera
### MEDV Mediana del valore di abitazioni occupate in 1.000 USD.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("regression").getOrCreate()

In [4]:
housing_df = spark.read.csv("./data/houses_train.csv", inferSchema=True, header=True)
housing_df.show(5)

+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
| ID|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|  1|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|  2|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|  4|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|  5|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|  7|0.08829|12.5| 7.87|   0|0.524|6.012|66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows



# Preprocessing dei dati

In [16]:
features_cols = housing_df.columns[1:-1]
print(features_cols)

['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat']


In [20]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=features_cols, outputCol="features")
data_df = assembler.transform(housing_df)
data_df.show(5)

+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
| ID|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|            features|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|  1|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|  2|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|  4|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|  5|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|  7|0.08829|12.5| 7.87|   0|0.524|6.012|66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|[0.08829,12.5,7.8...|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
only showing top 5 